In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model
import matplotlib.pyplot as plt



df = pd.read_excel(r'Treapta4TLXplus80Nm.xlsx', sheet_name='Restreint' )
dataset = df.values
X = dataset[1:,23:43]
Y = dataset[1:,14]
X = X.astype('float')
Y = Y.astype('float')

##0,12,4,16
d1 = pd.DataFrame(X)
d2 = pd.DataFrame(Y)
d1.drop([1,2,3,5,6,8,9,10,11,13,14,15,17,18], axis=1, inplace=True)
d2.replace(0.  , np.nan, inplace=True)
d2.replace(-1. , np.nan, inplace=True)
d2.replace(1.  , np.nan, inplace=True)
d2.replace(2.  , np.nan, inplace=True)
d2.replace(3.  , np.nan, inplace=True)
d2.replace(4.  , np.nan, inplace=True)
d2.replace(5.  , np.nan, inplace=True)
d2.replace(6.  , np.nan, inplace=True)


d1[5] = d2[0]
d1.dropna(inplace=True)
# d1.isnull().any() # Verifica daca avem vre-o valoare de tip NaN

X = d1.values[0:,:-1]
Y = d1.values[0:, -1:]


min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
Y_scaler = preprocessing.MinMaxScaler()
Y_scale = Y_scaler.fit_transform(Y)




X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape




def tree_model(x, dropout=0.2):
    model = Sequential()
    model.add(Dense(x, input_dim=6, kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/4),  kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def basic_model_3(x):
     # create model
    model = Sequential()
    model.add(Dense(x, input_dim=6, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(x, input_dim=4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(x, input_dim=2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def etree_model(x, dropout=0.2):
    model = Sequential()
    model.add(Dense(x, input_dim=6, kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x*2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/4),  kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


def err(model, x):
    emax = 1.
    emin = 1.
    for i in range(x):
        p = model.predict(X_train[i].reshape(1, 6))
        #print(p, Y_train[i], p/float(Y_train[i]))
        pp = Y_scaler.inverse_transform(p)
        py = Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
        r = p/float(Y_train[i])
        if r > emax and py > 7.:
            emax = r
        if r < emin and py > 7.:
            emin = r
        print(pp,py, r)
    return emin , emax;


def process_model(model, batch_size=32, epochs=500):
    keras_callbacks = [
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
                # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
                EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
                ]
    history = model.fit(X_train, Y_train, batch_size=batch_size,epochs=epochs,
                        shuffle=True, verbose=0, validation_data=(X_test, Y_test), callbacks=keras_callbacks)
                
    train_score = model.evaluate(X_train, Y_train, verbose=0)
    valid_score = model.evaluate(X_test, Y_test, verbose=0)
    
    
bs=64
rmin=10
rmax=100
for x4 in range(rmin,rmax):
    em = etree_model(x4, dropout=0.2)
    process_model(em, batch_size=64, epochs=5000)
    print(err(em, 50), bs, x4)


Using TensorFlow backend.
W1011 12:18:11.534667 12096 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1011 12:18:11.582128 12096 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1011 12:18:11.586128 12096 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W1011 12:18:11.621129 12096 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

[[5.046002]] [[0.19]] [[1.8594693]]
[[5.046002]] [[7.09]] [[0.8371316]]
[[5.046002]] [[7.]] [[0.84317833]]
[[5.046002]] [[10.]] [[0.67956024]]
[[5.046002]] [[6.81]] [[0.8562349]]
[[5.046002]] [[6.02]] [[0.91515696]]
[[5.046002]] [[6.46]] [[0.881376]]
[[5.046002]] [[4.21]] [[1.0864532]]
[[5.046002]] [[3.99]] [[1.1117462]]
[[5.046002]] [[3.23]] [[1.208976]]
[[5.046002]] [[10.]] [[0.67956024]]
[[5.046002]] [[5.91]] [[0.9240107]]
[[5.046002]] [[5.76]] [[0.9363638]]
[[5.046002]] [[1.38]] [[1.5359652]]
[[5.046002]] [[4.42]] [[1.0633606]]
[[5.046002]] [[2.85]] [[1.2642602]]
[[5.046002]] [[7.]] [[0.84317833]]
[[5.046002]] [[5.31]] [[0.97548765]]
[[5.046002]] [[3.81]] [[1.1333336]]
[[5.046002]] [[1.6]] [[1.4881022]]
[[5.046002]] [[7.86]] [[0.78873885]]
[[5.046002]] [[8.27]] [[0.76518583]]
[[5.046002]] [[4.92]] [[1.012139]]
[[5.046002]] [[5.95]] [[0.92077136]]
[[5.046002]] [[8.]] [[0.7805351]]
[[5.046002]] [[9.64]] [[0.69576174]]
[[5.046002]] [[5.32]] [[0.9745827]]
[[5.046002]] [[7.09]] [[0.8371

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[[4.2346087]] [[0.19]] [[1.7158599]]
[[4.2346087]] [[7.09]] [[0.7724788]]
[[4.710871]] [[7.]] [[0.8162818]]
[[8.203903]] [[10.]] [[0.883823]]
[[5.265989]] [[6.81]] [[0.87416375]]
[[4.2346087]] [[6.02]] [[0.84447813]]
[[5.715305]] [[6.46]] [[0.9375256]]
[[4.710871]] [[4.21]] [[1.0517964]]
[[5.0169444]] [[3.99]] [[1.1086714]]
[[4.2346087]] [[3.23]] [[1.1156051]]
[[7.3448453]] [[10.]] [[0.8282565]]
[[6.3164673]] [[5.91]] [[1.0357491]]
[[4.2346087]] [[5.76]] [[0.8640471]]
[[4.2346087]] [[1.38]] [[1.4173405]]
[[4.2346087]] [[4.42]] [[0.9812357]]
[[4.2346087]] [[2.85]] [[1.1666195]]
[[6.2976465]] [[7.]] [[0.94363135]]
[[5.874383]] [[5.31]] [[1.0524032]]
[[4.2346087]] [[3.81]] [[1.0458046]]
[[4.710871]] [[1.6]] [[1.4406333]]
[[4.2346087]] [[7.86]] [[0.7278235]]
[[5.842725]] [[8.27]] [[0.82321376]]
[[4.2346087]] [[4.92]] [[0.93397003]]
[[5.7054996]] [[5.95]] [[0.97857136]]
[[6.6362963]] [[8.]] [[0.89868474]]
[[5.59016]] [[9.64]] [[0.73179865]]
[[5.970888]] [[5.32]] [[1.0603791]]
[[4.2346087]] 

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


...

[[1.3679805]] [[0.19]] [[1.208492]]
[[7.470634]] [[7.09]] [[1.0303295]]
[[7.000469]] [[7.]] [[1.0000377]]
[[9.163337]] [[10.]] [[0.945882]]
[[6.5833507]] [[6.81]] [[0.9815282]]
[[5.6158824]] [[6.02]] [[0.9647981]]
[[6.0422783]] [[6.46]] [[0.9649563]]
[[4.183405]] [[4.21]] [[0.9972498]]
[[3.8951957]] [[3.99]] [[0.9899678]]
[[3.5381777]] [[3.23]] [[1.0354635]]
[[9.588715]] [[10.]] [[0.9733968]]
[[5.1177897]] [[5.91]] [[0.9303245]]
[[5.2815495]] [[5.76]] [[0.95735735]]
[[1.6104956]] [[1.38]] [[1.0336982]]
[[3.7812471]] [[4.42]] [[0.9353489]]
[[2.475093]] [[2.85]] [[0.9548848]]
[[7.6893277]] [[7.]] [[1.0553232]]
[[4.58756]] [[5.31]] [[0.9329211]]
[[3.8889768]] [[3.81]] [[1.0085196]]
[[2.3669834]] [[1.6]] [[1.1086378]]
[[7.1607347]] [[7.86]] [[0.94750255]]
[[8.240147]] [[8.27]] [[0.9978256]]
[[4.206913]] [[4.92]] [[0.93130183]]
[[6.228237]] [[5.95]] [[1.0243853]]
[[8.088263]] [[8.]] [[1.0065575]]
[[9.467247]] [[9.64]] [[0.98855937]]
[[4.763555]] [[5.32]] [[0.94838166]]
[[7.673961]] [[7.09]]

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[[0.8059649]] [[0.19]] [[1.1090202]]
[[7.3614225]] [[7.09]] [[1.0216273]]
[[7.113145]] [[7.]] [[1.0090806]]
[[8.611953]] [[10.]] [[0.9102168]]
[[6.762872]] [[6.81]] [[0.99615914]]
[[5.722664]] [[6.02]] [[0.97409964]]
[[5.652338]] [[6.46]] [[0.9322431]]
[[4.0761495]] [[4.21]] [[0.9861582]]
[[3.3186538]] [[3.99]] [[0.9289581]]
[[2.19157]] [[3.23]] [[0.8805029]]
[[9.270805]] [[10.]] [[0.9528335]]
[[5.1184497]] [[5.91]] [[0.93038255]]
[[5.349051]] [[5.76]] [[0.9633735]]
[[1.3276361]] [[1.38]] [[0.9923445]]
[[3.9230053]] [[4.42]] [[0.9496969]]
[[2.3313017]] [[2.85]] [[0.9375814]]
[[7.4095955]] [[7.]] [[1.0328728]]
[[4.662336]] [[5.31]] [[0.93986404]]
[[3.366418]] [[3.81]] [[0.9521486]]
[[1.8643929]] [[1.6]] [[1.0374494]]
[[7.189268]] [[7.86]] [[0.94964474]]
[[7.904146]] [[8.27]] [[0.9733537]]
[[4.9179835]] [[4.92]] [[0.99980575]]
[[6.4271955]] [[5.95]] [[1.0418226]]
[[8.09753]] [[8.]] [[1.007246]]
[[9.347508]] [[9.64]] [[0.9806297]]
[[5.087663]] [[5.32]] [[0.9784474]]
[[7.0526576]] [[7.09]]

C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_mean_absolute_error` which is not available. Available metrics are: val_loss,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


[[0.9526339]] [[0.19]] [[1.1349794]]
[[6.6664186]] [[7.09]] [[0.9662485]]
[[7.2153344]] [[7.]] [[1.017282]]
[[8.924433]] [[10.]] [[0.930429]]
[[6.598577]] [[6.81]] [[0.98276913]]
[[5.8511405]] [[6.02]] [[0.985291]]
[[6.4015417]] [[6.46]] [[0.9950958]]
[[4.0847764]] [[4.21]] [[0.9870503]]
[[3.673829]] [[3.99]] [[0.9665428]]
[[3.1296136]] [[3.23]] [[0.9884481]]
[[9.727076]] [[10.]] [[0.9823464]]
[[5.2451534]] [[5.91]] [[0.94152623]]
[[5.3535943]] [[5.76]] [[0.96377844]]
[[1.470995]] [[1.38]] [[1.0133034]]
[[4.11498]] [[4.42]] [[0.96912754]]
[[2.5996618]] [[2.85]] [[0.9698751]]
[[7.2253466]] [[7.]] [[1.0180856]]
[[4.9150853]] [[5.31]] [[0.963332]]
[[3.6749656]] [[3.81]] [[0.98543316]]
[[1.7035254]] [[1.6]] [[1.0146637]]
[[7.2732387]] [[7.86]] [[0.9559488]]
[[7.8537407]] [[8.27]] [[0.9696825]]
[[4.427268]] [[4.92]] [[0.9525306]]
[[6.4743176]] [[5.95]] [[1.0459524]]
[[7.929974]] [[8.]] [[0.9947975]]
[[9.171677]] [[9.64]] [[0.9689852]]
[[5.1529627]] [[5.32]] [[0.9845049]]
[[6.9078608]] [[7.0